In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json
from selenium import webdriver
from selenium.common.exceptions import WebDriverException

In [4]:
df = pd.read_csv('hs_info.csv', dtype={'id': str})
df

C:\Users\nguye\AppData\Local\Temp\ipykernel_8148\4193346499.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('hs_info.csv', dtype={'id': str})


,Unnamed: 0.1,Unnamed: 0,id,code,description,hasChildren,section,index,crawled,country,parent_id,error_message,selectable,intervalMin,intervalMax
0,0,0.0,-1,NaN,Live animals; animal products,True,"{'description': 'SECTION I', 'chapterFrom': '0...",0.0,Done,AT,NaN,NaN,NaN,NaN,NaN
1,1,1.0,-2,NaN,Vegetable products,True,"{'description': 'SECTION II', 'chapterFrom': '...",1.0,Done,AT,NaN,NaN,NaN,NaN,NaN
2,2,2.0,-3,NaN,Animal or vegetable fats and oils and their cl...,True,"{'description': 'SECTION III', 'chapterFrom': ...",2.0,Done,AT,NaN,NaN,NaN,NaN,NaN
3,3,3.0,-4,NaN,"Prepared foodstuffs; beverages, spirits and vi...",True,"{'description': 'SECTION IV', 'chapterFrom': '...",3.0,Done,AT,NaN,NaN,NaN,NaN,NaN
4,4,4.0,-5,NaN,Mineral products,True,"{'description': 'SECTION V', 'chapterFrom': '2...",4.0,Done,AT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76017,76017,NaN,61510,1.006310e+09,Other,False,NaN,1.0,NaN,BG,58310.0,NaN,True,NaN,NaN
76018,76018,NaN,103216,1.604145e+09,For processing,False,NaN,0.0,NaN,BG,99616.0,NaN,True,NaN,NaN
76019,76019,NaN,103316,1.604145e+09,Other,False,NaN,1.0,NaN,BG,99616.0,NaN,True,NaN,NaN
76020,76020,NaN,103416,1.604145e+09,For processing,False,NaN,0.0,NaN,BG,99716.0,NaN,True,NaN,NaN
